In [ ]:
import cv2
from PIL import Image
import sys
import pyocr
import pyocr.builders

In [ ]:
# 文字認識
def ocr(pic):
  tools = pyocr.get_available_tools()
  if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)
  tool = tools[0]

  result = []
  width = 275; height = 40 #切り出す画像のサイズ
  x = 675; y = 54 #基準点

  img = cv2.imread(pic, cv2.IMREAD_GRAYSCALE)

  for i in range(12):
    # 分割、2値化
    img_cropped = img[y:y+height, x:x+width]
    y = y + height + 12
    ret, img_thresh = cv2.threshold(img_cropped, 180, 255, cv2.THRESH_BINARY)

    if img_thresh[0][0] == 0:
      img_thresh = 255 - img_thresh

    # デバッグ用
    # cv2.imwrite("./data/save/" + str(i) + ".png", img_thresh)

    #文字認識
    new_image = Image.fromarray(img_thresh)
    txt = tool.image_to_string(
      new_image,
      lang="jpn+eng",
      builder = pyocr.builders.TextBuilder(tesseract_layout=7)
    )
    result.append(txt)

  return result

In [ ]:
# 得点計算
def calculate_score(pred_array, team_tag, enemy_tag):
  rank_array = []
  score = 0
  score_array = [15, 12, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

  # 順位の決定
  for i in range(12):
    if team_tag in pred_array[i]:
      rank_array.append(1)
    elif enemy_tag in pred_array[i]:
      rank_array.append(0)
    else: #どちらのタグも完全一致しない場合
      value = detailed_search(pred_array[i], team_tag, enemy_tag)
      rank_array.append(value)

  # 計算
  for i in range(12):
    if rank_array[i] == 1:
      score += score_array[i]

  print('【順位の表示】\n味方の順位を「1」で、敵の順位を「0」で表します。\n' + str(rank_array))
  print('\n')
  print('【得点の表示】\nこのレースで獲得した合計得点は「' + str(score) + '点」です。')
  print('-------------------------------------------------------------')

In [ ]:
def detailed_search(pred, team_tag, enemy_tag):
  if team_tag.lower() in pred.lower(): #大文字と小文字を区別しない
    value = 1
  elif enemy_tag.lower() in pred.lower():
    value = 0
  else:
    value = '?'
  return value

In [ ]:
pred_array = ocr("./data/pic/0.jpeg")
print(pred_array)
calculate_score(pred_array, "NvK", "RiZE")